# iOS人脸识别库（静态库）

## 创建静态库
![](images/create_ios_static.png)

## dlib库源代码添加到工程
* 把dlib拷贝到新建工程目录下

## 编译dlib库

* 使用Xcode构建dlib工程

```bash
cd dlib/dlib
mkdir build
cd build
cmake -G Xcode ..
```

* 打开工程文件dlib.xcodeproj进行配置

```
TARGETS -> dlib -> Build Settings -> Base SDK = iOS
TARGETS -> dlib -> Build Settings -> Build Active Architecture Only = No
TARGETS -> dlib -> Build Settings -> Supported Platforms = iOS
TARGETS -> dlib -> Build Settings -> Valid Architectures = arm64

设置活动的Scheme为dlib，Edit Scheme -> Run -> Build Configuration = Release
```

* 编译（Product -> Build）

```
通过设置活动的Scheme，编译设备和模拟器对应的静态库。
```

* 合并为一个静态库

```bash
lipo -create Debug-iphoneos/libdlib-ios.a Debug-iphonesimulator/libdlib-ios.a -output libdlib.a
```

## 拷贝libdlib.a到工程目录下的libs目录

## 工程配置

```
TARGETS -> face-recognition -> Build Settings -> Build Active Architecture Only = No
TARGETS -> face-recognition -> Build Settings -> Header Search Paths = dlib/
TARGETS -> face-recognition -> Build Settings -> Library Search Paths = libs/

TARGETS -> face-recognition -> Build Phases -> Link Binary With Libraries 添加 libdlib.a
TARGETS -> face-recognition -> Build Phases -> Copy Files -> Add -> face_recognition.hpp

TARGETS -> Add -> Aggregate -> distribution
TARGETS -> distribution -> Build Phases -> Add -> New Run Script Phase
TARGETS -> distribution -> Build Phases -> Run Script -> Shell 添加 ./build_distribution.sh

设置活动的Scheme为distribution，Edit Scheme -> Run -> Build Configuration = Release
```

## 创建用于自动发布的脚本

* 在工程目录下新建脚本文件：build_distribution.sh

```bash
#!/bin/sh

PROJ=${PROJECT_NAME}.xcodeproj
LIB_STATIC_NAME=${PROJECT_NAME}     #把项目名改为自己的即可使用
DISTRIBUTION_DIR=./distribution     #打包出来的文件的目录

IPHONE_OS_DIR=${DISTRIBUTION_DIR}/${CONFIGURATION}-iphoneos
IPHONE_SIMULATOR_DIR=${DISTRIBUTION_DIR}/${CONFIGURATION}-iphonesimulator

#创建真机库文件目录
if [[ ! -d ${IPHONE_OS_DIR} ]]; then
mkdir -p ${IPHONE_OS_DIR}
fi

#创建模拟器库文件目录
if [[ ! -d ${IPHONE_SIMULATOR_DIR} ]]; then
mkdir -p ${IPHONE_SIMULATOR_DIR}
fi


#error: error: accessing build database "/Users/wjj/Library/Developer/Xcode/DerivedData/face-recognition-ardscqnlcsvcvtgjntliogvokvae/Build/Intermediates.noindex/XCBuildData/build.db": disk I/O error
#-UseModernBuildSystem=NO 解决上面的问题（https://stackoverflow.com/questions/51153525/xcode-10-unable-to-attach-db-error）
#编译真机库文件
xcodebuild -project ${PROJ} \
-scheme ${LIB_STATIC_NAME} \
-configuration ${CONFIGURATION} \
-sdk iphoneos \
-UseModernBuildSystem=NO \
clean \
build \
CONFIGURATION_BUILD_DIR=${IPHONE_OS_DIR}
# -archivePath ${IPHONE_OS_DIR}

#编译模拟器库文件
xcodebuild build -project ${PROJ} \
-scheme ${LIB_STATIC_NAME} \
-configuration ${CONFIGURATION} \
-sdk iphonesimulator \
-UseModernBuildSystem=NO \
clean \
build \
CONFIGURATION_BUILD_DIR=${IPHONE_SIMULATOR_DIR}
# -archivePath ${IPHONE_SIMULATOR_DIR}

##建立SDK目录
SDK_DIR=${DISTRIBUTION_DIR}/${LIB_STATIC_NAME}
if [[ -d ${SDK_DIR} ]]; then
rm -fR ${SDK_DIR}
fi
mkdir -p ${SDK_DIR}

# 静态库文件
LIB_NAME=lib${LIB_STATIC_NAME}.a

#合并模拟器文件和真机文件
lipo -create ${IPHONE_OS_DIR}/${LIB_NAME} ${IPHONE_SIMULATOR_DIR}/${LIB_NAME} -output ${SDK_DIR}/${LIB_NAME}

#拷贝头文件
cp -R ${IPHONE_OS_DIR}/include/${LIB_STATIC_NAME}/* ${SDK_DIR}/


#打包为zip文件
PACKAGE_DATE=`date '+%Y%m%d%H'`

pushd ${DISTRIBUTION_DIR}
SDK_ZIP_NAME=iOS_${LIB_STATIC_NAME}_${PACKAGE_DATE}_${CONFIGURATION}.zip
zip -qr ${SDK_ZIP_NAME} ${LIB_STATIC_NAME}
popd ${DISTRIBUTION_DIR}
```

## 给脚本添加执行权限

```bash
$ sudo chmod +x build_distribution.sh
```

## 编译工程

## 参考资料
* [Cross-compile to static lib (libgcrypt) for use on iOS](https://stackoverflow.com/questions/26812060/cross-compile-to-static-lib-libgcrypt-for-use-on-ios)
* [iOS 制作静态库自动编译脚本](https://www.jianshu.com/p/c796fcba1604)
* [静态库自动打包合并静态库脚本](https://blog.csdn.net/zxw_xzr/article/details/79217516)
* [iOS编译OpenSSL静态库(使用脚本自动编译)](https://www.jianshu.com/p/651513cab181)
* [iOS 静态库打包流程简化](https://blog.csdn.net/zhouzhoujianquan/article/details/53192597)
* [linux 下shell中if的“-e，-d，-f”是什么意思](https://www.cnblogs.com/senior-engineer/p/6206329.html)
* [Linux命令-cp 把整个目录下文件复制到另一个目录](https://blog.csdn.net/sdtvyyb_007/article/details/53423182)
* [Compile Collada DOM (c++) for use in iOS](https://stackoverflow.com/questions/30810809/compile-collada-dom-c-for-use-in-ios)
* [How to set up CMake to build a library for the iPhone](https://stackoverflow.com/questions/794137/how-to-set-up-cmake-to-build-a-library-for-the-iphone)
* [ios-cmake](https://github.com/sheldonth/ios-cmake/blob/master/ios.cmake)
* [How to set up CMake to build an app for the iPhone](https://stackoverflow.com/questions/822404/how-to-set-up-cmake-to-build-an-app-for-the-iphone)
* [How can I make Cmake use specific compiler and flags when final compilation stage instead of detection?](https://stackoverflow.com/questions/6476203/how-can-i-make-cmake-use-specific-compiler-and-flags-when-final-compilation-stag)
* [Xcode7 制作Framework](https://www.cnblogs.com/developer-qin/p/5729250.html)
* [iOS framework](http://www.cnblogs.com/developer-qin/p/5691008.html)
* [Xcode 6制作动态及静态Framework](http://www.cocoachina.com/ios/20141126/10322.html)
* [Xcode 10: unable to attach DB error](https://stackoverflow.com/questions/51153525/xcode-10-unable-to-attach-db-error)
* [Xcode 10 beta error while building my project?](https://stackoverflow.com/questions/51562894/xcode-10-beta-error-while-building-my-project)
* [用lipo合并模拟器Framework与真机Framework](https://blog.csdn.net/gavin8803/article/details/52103671)
* [iOS中的静态库与动态库，区别、制作和使用](http://www.cocoachina.com/ios/20161109/18031.html)
* [Xcode7 制作通用的framework，静态库和动态库](https://www.jianshu.com/p/2b04935d6943)